In [ ]:
import argparse
import os
import numpy as np
import math

from torchvision import transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
os.makedirs('images', exist_ok=True)

In [ ]:
# 주피터 노트북에서 실행시키기 위한 args class생성

class args:
    def __init__(self, n_epochs=200, batch_size=64, lr=0.0002, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, num_classes=10, img_size=32, channels=1, sample_interval=400):
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.lr =lr
        self.b1 = b1
        self.b2 = b2
        self.n_cpu = n_cpu
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.img_size = img_size
        self.channels = channels
        self.sample_interval = sample_interval

In [ ]:
args = args()

In [ ]:
img_shape = (args.channels, args.img_size, args.img_size)

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print(f'There are {torch.cuda.device_count()} GPU(s) available.')

    print(f'We will use the GPU: {torch.cuda.get_device_name(0)}')

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(args.num_classes, args.num_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(args.latent_dim + args.num_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(args.num_classes, args.num_classes)

        self.model = nn.Sequential(
            nn.Linear(args.num_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1)
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to product input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

In [ ]:
# Loss function
adversarial_loss = torch.nn.MSELoss()

In [ ]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [ ]:
generator.to(device)
discriminator.to(device)
adversarial_loss.to(device)

MSELoss()

In [ ]:
# Configure data laoder
datasets.MNIST.resources = [
            ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
            ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c'), 
        ]
        
os.makedirs('./data/mnist', exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        './data/mnist',
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(args.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=args.batch_size,
    shuffle =True,
)

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=args.lr, betas=(args.b1, args.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=args.lr, betas=(args.b1, args.b2))

In [ ]:
FloatTensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor

In [ ]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, args.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, n_row=n_row, normalize=True)

In [ ]:
# Training
for epoch in range(args.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # Train Generator
        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, args.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, args.num_classes, batch_size)))
        
        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measure Generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # Train discriminator
        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % args.sample_interval == 0:
            save_image(gen_imgs.data[:25], 'images/%d.png' % batches_done, nrow=5, normalize=True)
        
    print(f'[Epoch {epoch}/{args.n_epochs}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]')

[Epoch 0/200] [D loss: 0.1172456443309784] [G loss: 0.4900827705860138]
[Epoch 1/200] [D loss: 0.14721912145614624] [G loss: 0.2921856641769409]
[Epoch 2/200] [D loss: 0.1065777987241745] [G loss: 0.4988553524017334]
[Epoch 3/200] [D loss: 0.13389690220355988] [G loss: 0.3984091281890869]
[Epoch 4/200] [D loss: 0.15800873935222626] [G loss: 0.6341167688369751]
[Epoch 5/200] [D loss: 0.15028591454029083] [G loss: 0.5738266110420227]
[Epoch 6/200] [D loss: 0.17589208483695984] [G loss: 0.469452440738678]
[Epoch 7/200] [D loss: 0.17452290654182434] [G loss: 0.4780464768409729]
[Epoch 8/200] [D loss: 0.21498951315879822] [G loss: 0.350124329328537]
[Epoch 9/200] [D loss: 0.20897561311721802] [G loss: 0.36765241622924805]
[Epoch 10/200] [D loss: 0.2157338261604309] [G loss: 0.34814608097076416]
[Epoch 11/200] [D loss: 0.20554566383361816] [G loss: 0.31542661786079407]
[Epoch 12/200] [D loss: 0.2036687731742859] [G loss: 0.3276912569999695]
[Epoch 13/200] [D loss: 0.21300190687179565] [G los

In [ ]:
import cv2
from pathlib import Path
def count_data(path, label):
    path=Path(PATH + label)
    #path=path.glob("*.jpg")
    images=[]
    cnt = 0
    for imagepath in path.glob('*.png'):
        cnt += 1

    return cnt    

In [ ]:
PATH = '/content/images/'

In [ ]:
count_data(PATH, '')

469

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

for idx in range(0, count_data(PATH, ''), 10):
    name = str(idx * args.sample_interval) + '.png'
    img = Image.open(PATH+name)
    plt.imshow(img)
    plt.title(name)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.